In [1]:
# GET /hello_world

import json
print(json.dumps({
  'hello': 'world'
}))

{"hello": "world"}


## Template for event trigger to ETL data

In [ ]:
# POST /handle_event

GRAPHQL_ENDPOINT = ""
ADMIN_SECRET = ""
OPENAI_API_KEY = ""


import json
import openai
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

openai.api_key = OPENAI_API_KEY


def get_embedding(text_to_embed):
	# Embed a line of text
	response = openai.Embedding.create(
    	model= "text-embedding-ada-002",
    	input=[text_to_embed]
	)
	# Extract the AI output embedding as a list of floats
	embedding = response["data"][0]["embedding"]
	return embedding

def handle_insert(row, client):
    book_id = row['id']
    description = row['description']
    embedding = get_embedding(description)

    gql_query = gql("""
            mutation insertEmbeddings($id: bigint!, $embedding: String!) {
                insert_books_embedding(args: { book_id: $id, embedding: $embedding }) {
                    id
                }
            }
        """)
    print(client.execute(gql_query, variable_values={
        'id': str(book_id), 'embedding': str(embedding)}))


def handle_event(request):
    print("handle_event function got called")
    gql_headers = {'x-hasura-admin-secret': ADMIN_SECRET}
    # Create a GraphQL client with the request transport
    transport = RequestsHTTPTransport(
        url=GRAPHQL_ENDPOINT, headers=gql_headers)
    client = Client(transport=transport)

    event = request['body']['event']
    op = event['op']
    if op == 'INSERT':
        row = event['data']['new']
        handle_insert(row, client)
    else:
        print(str(event))
    return request

print(json.dumps(handle_event(json.loads(REQUEST))))

In [ ]:
# POST /handle_query

GRAPHQL_ENDPOINT = ""
ADMIN_SECRET = ""
OPENAI_API_KEY = ""


import json
import openai
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


openai.api_key = OPENAI_API_KEY


def get_embedding(text_to_embed):
	# Embed a line of text
	response = openai.Embedding.create(
    	model= "text-embedding-ada-002",
    	input=[text_to_embed]
	)
	# Extract the AI output embedding as a list of floats
	embedding = response["data"][0]["embedding"]
	return embedding

def handle_query(request):
    user_query = request['body']['input']['user_query']

    gql_headers = dict()
    gql_headers['x-hasura-admin-secret'] = ADMIN_SECRET

    # Create a GraphQL client with the request transport
    transport = RequestsHTTPTransport(
        url=GRAPHQL_ENDPOINT, headers=gql_headers)
    client = Client(transport=transport)

    query_embedding = get_embedding(user_query)
    # Send the GraphQL request
    gql_query = gql("""
            query getSimilarBooks($query_embedding: String!) {
                find_books_by_desc_similarity(args: { book_query_embedding: $query_embedding }) {
                    id
                    name
                    genre
                    description
                }
            }
        """)
    result = client.execute(gql_query, variable_values={
                            'query_embedding': str(query_embedding)})
    
    prompt = """
    You are helpful book recommendation bot. You are given a user query and a list of books.
    Use the following context to recommend the best books to the user and explain why.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    \n
    """
    prompt += user_query

    books = result["find_books_by_desc_similarity"]

    for b in books:
        prompt += "Book Name:"
        prompt += b["name"]
        prompt += "Book Description: "
        prompt += b["description"]
        prompt += "Book Genre: "
        prompt += b["genre"]
        prompt += "\n"

    prompt += "\nQuestion: {question}"

    llm = OpenAI(model="text-davinci-003",
                openai_api_key=OPENAI_API_KEY)
    chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt))

    return chain.run({"question":user_query})

print(json.dumps(handle_query(json.loads(REQUEST))))